In [ ]:
pip install pyexiftool

In [ ]:
pip install pyproj

In [18]:
pip install shapely

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
    --------------------------------------- 0.0/1.4 MB 640.0 kB/s eta 0:00:03
   - -------------------------------------- 0.1/1.4 MB 787.7 kB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.4 MB 2.0 MB/s eta 0:00:01
   -------------- ------------------------- 0.5/1.4 MB 3.3 MB/s eta 0:00:01
   ----------------------- ---------------- 0.9/1.4 MB 3.9 MB/s eta 0:00:01
   -------------------------------- ------- 1.2/1.4 MB 4.4 MB/s eta 0:00:01
   ---------------------------------------  1.4/1.4 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Read Exifdata from UAV-Images

In [28]:
import os
import math
import re
import subprocess

# Path to the folder containing the images
folder_path = r'C:\Users\felix\Documents\My_Python\data\Testzone\turn'
# Path to all images in the folder
#folder_path = r"E:\Data_MaterThesis\Data_Ashaiman\Raw_Images\Zone_14\Zone_14"


# Path to the ExifTool executable
exifToolPath = "exiftool.exe"  # Adjust if necessary

# Initialize the dictionary to store metadata
image_metadata = {}

# Filter for image files in the folder (common extensions)
valid_extensions = {'.jpg', '.jpeg', '.png'}
image_files = [f for f in os.listdir(folder_path) if os.path.splitext(f)[-1].lower() in valid_extensions]

# Process each image in the folder
for filename in image_files:
    image_path = os.path.join(folder_path, filename)
    
    # Initialize subprocess to call ExifTool
    process = subprocess.Popen([exifToolPath, image_path], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

    # Read and parse metadata
    infoDict = {}
    for tag in process.stdout:
        line = tag.strip().split(':')  # Split each line at the ':'
        if len(line) > 1:  # Ensure valid key-value pair
            infoDict[line[0].strip()] = line[-1].strip()

    # Extract required metadata
    try:
        relAlt = float(infoDict.get("Relative Altitude", 0))  # Default to 0 if not found
        #print(relAlt)
        absAlt = float(infoDict.get("Absolute Altitude", 0))
        gpslat = infoDict.get("GPS Latitude", "")
        gpslon = infoDict.get("GPS Longitude", "")
        iw = float(infoDict.get("Image Width", 0))
        #print(iw)
        fov = infoDict.get("Field Of View", "0")
        #print(fov)
        gimbal_roll_deg = float(infoDict.get("Gimbal Roll Degree", 0))
        gimbal_yaw_deg = float(infoDict.get("Gimbal Yaw Degree", 0))
        gimbal_pitch_deg = float(infoDict.get("Gimbal Pitch Degree", 0))
        flight_yaw_deg = float(infoDict.get("Flight Yaw Degree", 0))

        
        # Parse latitude
        if gpslat:
            lat_ori = gpslat[-1]
            matches = re.findall(r'(\d+(\.\d+)?)', gpslat)
            gpslat = [float(match[0]) for match in matches]
            lat_deg, lat_min, lat_sec = gpslat
            lat_dec = abs(lat_deg) + lat_min / 60 + lat_sec / 3600
            if lat_ori == 'S':
                lat_dec = -lat_dec
        else:
            lat_dec = None

        # Parse longitude
        if gpslon:
            lon_ori = gpslon[-1]
            matches = re.findall(r'(\d+(\.\d+)?)', gpslon)
            gpslon = [float(match[0]) for match in matches]
            lon_deg, lon_min, lon_sec = gpslon
            lon_dec = abs(lon_deg) + lon_min / 60 + lon_sec / 3600
            if lon_ori == 'W':
                lon_dec = -lon_dec
        else:
            lon_dec = None

        # Calculate GSD
        if iw > 0 and relAlt < 88:
            fov_value = float(re.findall(r'\d+\.\d+', fov)[0]) if fov else 0
            fovrad = math.radians(fov_value / 2)
            gsd = (math.tan(fovrad) * relAlt) / (iw / 2)
        else:
            gsd = 0.022 # Default value

        # Store the data in a dictionary
        image_metadata[filename] = {
            'Latitude': lat_dec,
            'Longitude': lon_dec,
            'Relative Altitude': relAlt,
            'Absolute Altitude': absAlt,
            'GSD': gsd,
            "Gimbal Roll Degree": gimbal_roll_deg,
            "Gimbal Yaw Degree": gimbal_yaw_deg,
            "Gimbal Pitch Degree": gimbal_pitch_deg,
            "Flight Yaw Degree": flight_yaw_deg
        }
    except Exception as e:
        print(f"Error processing file {filename}: {e}")

# Output the collected metadata
for image_name, metadata in image_metadata.items():
    print(f"Image: {image_name}")
    for key, value in metadata.items():
        print(f"  {key}: {value}")
    print()


Image: DJI_0069.JPG
  Latitude: 5.702191666666667
  Longitude: -0.01436388888888889
  Relative Altitude: 90.0
  Absolute Altitude: 163.27
  GSD: 0.022
  Gimbal Roll Degree: 0.0
  Gimbal Yaw Degree: -5.5
  Gimbal Pitch Degree: -90.0
  Flight Yaw Degree: -8.0

Image: DJI_0070.JPG
  Latitude: 5.7023222222222225
  Longitude: -0.014380555555555556
  Relative Altitude: 90.0
  Absolute Altitude: 163.3
  GSD: 0.022
  Gimbal Roll Degree: 0.0
  Gimbal Yaw Degree: -5.5
  Gimbal Pitch Degree: -89.9
  Flight Yaw Degree: -9.2

Image: DJI_0071.JPG
  Latitude: 5.702352777777778
  Longitude: -0.014408333333333332
  Relative Altitude: 89.9
  Absolute Altitude: 163.27
  GSD: 0.022
  Gimbal Roll Degree: 0.0
  Gimbal Yaw Degree: -5.5
  Gimbal Pitch Degree: -90.0
  Flight Yaw Degree: -24.2

Image: DJI_0072.JPG
  Latitude: 5.702194444444444
  Longitude: -0.014616666666666667
  Relative Altitude: 90.0
  Absolute Altitude: 163.9
  GSD: 0.022
  Gimbal Roll Degree: 0.0
  Gimbal Yaw Degree: 175.2
  Gimbal Pitch D

### Save output to CSV

In [11]:
import csv

# Function to save image metadata to a CSV file
def save_metadata_to_csv(image_metadata, output_file):
    # Open the CSV file for writing
    with open(output_file, mode='w', newline='', encoding='utf-8') as csvfile:
        # Create a CSV writer
        writer = csv.writer(csvfile)
        
        # Write the header row
        # Use the keys from the first image's metadata as the header
        header = ['Image'] + list(next(iter(image_metadata.values())).keys())
        writer.writerow(header)
        
        # Write the metadata rows
        for image, metadata in image_metadata.items():
            # Combine the image name and metadata values
            row = [image] + [metadata[key] for key in metadata]
            writer.writerow(row)

# Example usage
output_csv = 'image_metadata_zone14.csv'  # Specify the output file name
save_metadata_to_csv(image_metadata, output_csv)
print(f"Metadata saved to {output_csv}")

Metadata saved to image_metadata_zone14.csv


### Calculate Drone Movement between Pictures

In [ ]:
import math
from pyproj import Proj, transform


# Function to calculate pixel differences with middle point correction and yaw adjustments
def calculate_pixel_differences_with_middle_point_correction(image_metadata):
    # Sort images by their filenames to ensure correct order
    sorted_images = sorted(image_metadata.keys())
    results = []

    for i in range(len(sorted_images) - 1):
        # Get metadata for consecutive images
        img1 = sorted_images[i]
        img2 = sorted_images[i + 1]

        meta1 = image_metadata[img1]
        meta2 = image_metadata[img2]

        # Extract latitude, longitude, GSD, gimbal, flight angles, and altitude
        lat1, lon1, gsd1, altitude1 = meta1['Latitude'], meta1['Longitude'], meta1['GSD'], meta1['Relative Altitude']
        lat2, lon2, gsd2, altitude2 = meta2['Latitude'], meta2['Longitude'], meta2['GSD'], meta2['Relative Altitude']

        gimbal_pitch1, gimbal_yaw1, flight_yaw1 = (
            meta1.get('Gimbal Pitch Degree', -90),  # Default to downward (-90)
            meta1.get('Gimbal Yaw Degree', 0),
            meta1.get('Flight Yaw Degree', 0)
        )
        gimbal_pitch2, gimbal_yaw2, flight_yaw2 = (
            meta2.get('Gimbal Pitch Degree', -90),
            meta2.get('Gimbal Yaw Degree', 0),
            meta2.get('Flight Yaw Degree', 0)
        )

        # Check if metadata is valid
        if None in (lat1, lon1, gsd1, altitude1, lat2, lon2, gsd2, altitude2):
            print(f"Skipping image pair {img1} and {img2} due to missing data.")
            continue

        # Average GSD and altitude for the two images
        avg_gsd = (gsd1 + gsd2) / 2
        avg_altitude = (altitude1 + altitude2) / 2

        # Create a projection object for WGS84 (lat/lon) to UTM (zone determined automatically)
        utm_proj_1 = Proj(proj="utm", zone=32, ellps="WGS84")  # Use appropriate UTM zone for your location
        utm_x1, utm_y1 = utm_proj_1(lon1, lat1) 
        utm_proj_2 = Proj(proj="utm", zone=32, ellps="WGS84")  # Use appropriate UTM zone for your location
        utm_x2, utm_y2 = utm_proj_2(lon2, lat2)
        print(f"UTM Coordinates: X = {utm_x1}, Y = {utm_y1}")
        print(f"UTM Coordinates: X = {utm_x2}, Y = {utm_y2}")

        x_utm_diff = utm_x2 - utm_x1
        y_utm_diff = utm_y2 - utm_y1
        print(f"UTM Differences: dX = {x_utm_diff}, dY = {y_utm_diff}")

        # Convert ground distances to pixel distances
        delta_x_pixels = x_utm_diff / avg_gsd
        delta_y_pixels = y_utm_diff / avg_gsd
        print("delta_x_pixels",delta_x_pixels)
        print("delta_y_pixels",delta_y_pixels)

        # Average gimbal and flight yaw
        avg_gimbal_yaw = (gimbal_yaw1 + gimbal_yaw2) / 2
        avg_flight_yaw = (flight_yaw1 + flight_yaw2) / 2 

        # Combined yaw adjustment
        avg_total_yaw = avg_gimbal_yaw + avg_flight_yaw
        print("avg_total_yaw",avg_total_yaw)

        # Adjust pixel differences for yaw
        delta_x_corrected = delta_x_pixels * math.cos(math.radians(avg_total_yaw)) - delta_y_pixels * math.sin(math.radians(avg_total_yaw))
        delta_y_corrected = delta_x_pixels * math.sin(math.radians(avg_total_yaw)) + delta_y_pixels * math.cos(math.radians(avg_total_yaw))
        print("delta_x_corrected",delta_x_corrected)
        print("delta_y_corrected",delta_y_corrected)


        # Store results
        results.append({
            'image1': img1,
            'image2': img2,
            'delta_x_pixels': delta_x_corrected,
            'delta_y_pixels': delta_y_corrected
        })

    return results

        

# Use the metadata from the first program
# Assuming `image_metadata` was generated by the first program
pixel_differences = calculate_pixel_differences_with_middle_point_correction(image_metadata)

# Print results
for diff in pixel_differences:
    print(diff)


### Cross Correlation

In [35]:
import os
import numpy as np
from skimage.registration import phase_cross_correlation
from skimage import io
import math
from skimage.transform import rotate

def preprocess_image(image):
    """Preprocess the image by converting it to grayscale and normalizing."""
    if image.ndim == 3:
        image = image.mean(axis=2)  # Convert to grayscale by averaging color channels
    return image.astype(np.float32) / 255.0  # Normalize the image

def rotate_shift(shift, yaw):
    """Apply yaw rotation to the shift vector."""
    yaw_rad = math.radians(yaw)
    rotation_matrix = np.array([
        [math.cos(yaw_rad), -math.sin(yaw_rad)],
        [math.sin(yaw_rad), math.cos(yaw_rad)]
    ])
    return rotation_matrix.dot(shift)

def estimate_shift_and_rotation(image1, image2, yaw1, yaw2):
    """Estimate shift and detect rotation between two images."""
    image1 = preprocess_image(image1)
    image2 = preprocess_image(image2)

    # Compute the initial pixel shift using cross-correlation
    shift, _, _ = phase_cross_correlation(image1, image2, upsample_factor=10)

    # Detect if there’s a direction change
    yaw_diff = yaw2 - yaw1
    rotation_detected = abs(yaw_diff) > 1  # Small threshold to account for noise
    corrected_shift = shift

    rotation_angle = 0.0
    if rotation_detected:
        # Estimate the rotation angle based on yaw difference
        rotation_angle = yaw_diff
        
        # Rotate the second image for alignment
        image2_rotated = rotate(image2, angle=-rotation_angle, resize=False)
        
        # Recompute the shift after rotation correction
        corrected_shift, _, _ = phase_cross_correlation(image1, image2_rotated, upsample_factor=10)

    # Apply yaw correction to the shift vector
    corrected_shift = rotate_shift(corrected_shift, yaw_diff)

    return corrected_shift, rotation_angle

def process_image_folder_with_rotation(folder_path, image_metadata):
    """
    Process all images in the folder, estimating shifts and rotations between consecutive pairs.
    
    Args:
        folder_path (str): Path to the folder containing images.
        image_metadata (dict): Metadata for images, including yaw angles.
        
    Returns:
        dict: Dictionary containing shifts and rotations for each image pair.
    """
    images = sorted([f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.JPG', '.png', '.jpeg'))])
    results = {}

    for i in range(len(images) - 1):
        img1_path = os.path.join(folder_path, images[i])
        img2_path = os.path.join(folder_path, images[i + 1])

        # Load the images
        image1 = io.imread(img1_path)
        image2 = io.imread(img2_path)

        if image1 is None or image2 is None:
            print(f"Error loading images: {img1_path}, {img2_path}")
            continue

        # Get yaw angles from metadata
        yaw1 = image_metadata.get(images[i], {}).get('Gimbal Yaw Degree', 0)
        yaw2 = image_metadata.get(images[i + 1], {}).get('Gimbal Yaw Degree', 0)
        
        # Estimate shift and rotation
        shift, rotation_angle = estimate_shift_and_rotation(image1, image2, yaw1, yaw2)
        
        # Store the result in the dictionary
        results[f"{images[i]}_{images[i+1]}"] = {
            "shift": {"y": shift[0], "x": shift[1]},
            "rotation_angle": rotation_angle
        }
        
        print(f"Between {images[i]} and {images[i + 1]}:")
        print(f"  Shift (y, x): {shift[0]:.2f}, {shift[1]:.2f}")
        if rotation_angle != 0.0:
            print(f"  Rotation detected: {rotation_angle:.2f} degrees")
    
    return results

# Example usage
folder_path = r"C:\Users\felix\Documents\My_Python\data\Testzone\turn"

# Process folder and save results
results = process_image_folder_with_rotation(folder_path, image_metadata)

shifts_and_rotations = {
    pair: {
        'shift': data['shift'],
        'rotation_angle': data['rotation_angle']
    }
    for pair, data in results.items()
}

# Print results dictionary
print("\nShifts and Rotations:")
for pair, data in results.items():
    print(f"{pair}: Shift (y, x) = {data['shift']}, Rotation = {data['rotation_angle']:.2f}°")

#worked 18.4 sec for 5 pairs of images


Between DJI_0069.JPG and DJI_0070.JPG:
  Shift (y, x): -674.80, -33.10
Between DJI_0070.JPG and DJI_0071.JPG:
  Shift (y, x): -228.30, -112.30
Between DJI_0071.JPG and DJI_0072.JPG:
  Shift (y, x): -343.11, 1054.29
  Rotation detected: 180.70 degrees
Between DJI_0072.JPG and DJI_0073.JPG:
  Shift (y, x): -619.52, 94.25
Between DJI_0073.JPG and DJI_0074.JPG:
  Shift (y, x): -659.73, 108.10

Shifts and Rotations:
DJI_0069.JPG_DJI_0070.JPG: Shift (y, x) = {'y': -674.7999877929688, 'x': -33.099998474121094}, Rotation = 0.00°
DJI_0070.JPG_DJI_0071.JPG: Shift (y, x) = {'y': -228.3000030517578, 'x': -112.30000305175781}, Rotation = 0.00°
DJI_0071.JPG_DJI_0072.JPG: Shift (y, x) = {'y': -343.1058432502932, 'x': 1054.286981835931}, Rotation = 180.70°
DJI_0072.JPG_DJI_0073.JPG: Shift (y, x) = {'y': -619.5150284847455, 'x': 94.24505016999579}, Rotation = 0.00°
DJI_0073.JPG_DJI_0074.JPG: Shift (y, x) = {'y': -659.7266431006103, 'x': 108.10353739590184}, Rotation = 0.00°


### BoundingBox Matching

In [36]:
import csv
import numpy as np
from shapely.geometry import box

def load_bounding_boxes(csv_file):
    """Load bounding boxes from a CSV file."""
    bounding_boxes = {}
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            image = row['image']
            bbox = {
                'x': float(row['x']),
                'y': float(row['y']),
                'width': float(row['width']),
                'height': float(row['height']),
                'class': row['class'],
                'class_id': int(row['class_id']),
                'confidence': float(row['confidence']),
            }
            if image not in bounding_boxes:
                bounding_boxes[image] = []
            bounding_boxes[image].append(bbox)
    return bounding_boxes

def adjust_bounding_box(bbox, shift, rotation_angle, image_shape):
    """Adjust bounding box coordinates based on shift and rotation."""
    # Extract shift and rotation
    shift_y, shift_x = shift
    rotation_rad = np.radians(rotation_angle)

    # Center of the image for rotation
    cx, cy = image_shape[1] / 2, image_shape[0] / 2

    # Original bounding box corners
    x_min = bbox['x']
    y_min = bbox['y']
    x_max = bbox['x'] + bbox['width']
    y_max = bbox['y'] + bbox['height']

    # Adjust for shift
    x_min += shift_x
    x_max += shift_x
    y_min += shift_y
    y_max += shift_y

    # Adjust for rotation (rotate around the image center)
    corners = np.array([
        [x_min, y_min],
        [x_max, y_min],
        [x_max, y_max],
        [x_min, y_max]
    ])
    rotated_corners = []
    for x, y in corners:
        # Translate to origin (image center)
        x_centered = x - cx
        y_centered = y - cy

        # Apply rotation
        x_rotated = x_centered * np.cos(rotation_rad) - y_centered * np.sin(rotation_rad)
        y_rotated = x_centered * np.sin(rotation_rad) + y_centered * np.cos(rotation_rad)

        # Translate back
        rotated_corners.append((x_rotated + cx, y_rotated + cy))

    rotated_corners = np.array(rotated_corners)
    new_x_min, new_y_min = rotated_corners.min(axis=0)
    new_x_max, new_y_max = rotated_corners.max(axis=0)

    return new_x_min, new_y_min, new_x_max - new_x_min, new_y_max - new_y_min

def compute_iou(bbox1, bbox2):
    """Compute Intersection over Union (IoU) between two bounding boxes."""
    box1 = box(bbox1[0], bbox1[1], bbox1[0] + bbox1[2], bbox1[1] + bbox1[3])
    box2 = box(bbox2[0], bbox2[1], bbox2[0] + bbox2[2], bbox2[1] + bbox2[3])
    intersection = box1.intersection(box2).area
    union = box1.union(box2).area
    return intersection / union

def match_bounding_boxes(bboxes1, bboxes2, shift, rotation_angle, image_shape, iou_threshold=0.5):
    """Match bounding boxes between two images."""
    matched_pairs = []
    for bbox1 in bboxes1:
        adjusted_bbox1 = adjust_bounding_box(bbox1, shift, rotation_angle, image_shape)
        for bbox2 in bboxes2:
            bbox2_coords = (bbox2['x'], bbox2['y'], bbox2['width'], bbox2['height'])
            iou = compute_iou(adjusted_bbox1, bbox2_coords)
            if iou >= iou_threshold:
                matched_pairs.append((bbox1, bbox2, iou))
    return matched_pairs

def process_bounding_boxes(csv_file, shifts_and_rotations, image_shape, iou_threshold=0.5):
    """Process bounding boxes across all image pairs."""
    bounding_boxes = load_bounding_boxes(csv_file)
    results = []

    images = sorted(bounding_boxes.keys())
    for i in range(len(images) - 1):
        img1 = images[i]
        img2 = images[i + 1]

        bboxes1 = bounding_boxes[img1]
        bboxes2 = bounding_boxes[img2]

        shift = shifts_and_rotations.get((img1, img2), {}).get('shift', (0, 0))
        rotation_angle = shifts_and_rotations.get((img1, img2), {}).get('rotation_angle', 0)

        matched_pairs = match_bounding_boxes(bboxes1, bboxes2, shift, rotation_angle, image_shape, iou_threshold)

        results.append({
            'image_pair': (img1, img2),
            'matched_pairs': matched_pairs
        })

    return results

# Example usage
#Shifts and Rotations:
#DJI_0069.JPG_DJI_0070.JPG: Shift (y, x) = {'y': -674.7999877929688, 'x': -33.099998474121094}, Rotation = 0.00°
#DJI_0070.JPG_DJI_0071.JPG: Shift (y, x) = {'y': -228.3000030517578, 'x': -112.30000305175781}, Rotation = 0.00°
#DJI_0071.JPG_DJI_0072.JPG: Shift (y, x) = {'y': -343.1058432502932, 'x': 1054.286981835931}, Rotation = 180.70°
#DJI_0072.JPG_DJI_0073.JPG: Shift (y, x) = {'y': -619.5150284847455, 'x': 94.24505016999579}, Rotation = 0.00°
#DJI_0073.JPG_DJI_0074.JPG: Shift (y, x) = {'y': -659.7266431006103, 'x': 108.10353739590184}, Rotation = 0.00°

csv_file = r"C:\Users\felix\Documents\My_Python\data\Detect"

image_shape = (3648, 5472)  # Example image shape (height, width)

results = process_bounding_boxes(csv_file, shifts_and_rotations, image_shape)

# Print results
for result in results:
    print(f"Image Pair: {result['image_pair']}")
    for match in result['matched_pairs']:
        bbox1, bbox2, iou = match
        print(f"  Matched with IoU {iou:.2f}: {bbox1} -> {bbox2}")


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\felix\\Documents\\My_Python\\data\\Detect'